In [7]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.stats import norm

def analyze_asset(ticker, target_year):
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=2000)

        # Fetch historical asset data
        asset_data = yf.download(ticker, start=start_date, end=end_date)
        asset_data.dropna(inplace=True)

        # Calculate log returns
        asset_data['LogReturn'] = np.log(asset_data['Close']).shift(-1) - np.log(asset_data['Close'])
        asset_data.dropna(inplace=True)

        # Calculate mean and standard deviation of log returns
        mu = asset_data['LogReturn'].mean()
        sigma = asset_data['LogReturn'].std(ddof=1)

        # Calculate the number of trading days until the end of the target year
        current_date = datetime.now()
        target_date = datetime(target_year, 12, 31)
        trading_days_until_target = (target_date - current_date).days * (252 / 365)  # approx 252 trading days per year

        # Adjust mu and sigma for the time horizon
        projected_mu = mu * trading_days_until_target
        projected_sigma = sigma * np.sqrt(trading_days_until_target)

        # Calculate percentage change range with 95% confidence
        z_score = norm.ppf(0.975)  # 95% confidence z-score
        pct_change_range = (projected_mu - z_score * projected_sigma, projected_mu + z_score * projected_sigma)
        pct_change_range_percent = (np.exp(pct_change_range[0]) - 1, np.exp(pct_change_range[1]) - 1)  # convert log returns to percentage

        print(f'The expected percentage change range with 95% confidence for {ticker} by {target_year} is {pct_change_range_percent}')
        
        return pct_change_range_percent

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def main():
    # Get user input for the ticker and target year
    ticker = input("Enter the asset ticker (e.g., AAPL, BTC-USD, VTI): ").strip()
    target_year = int(input("Enter the target year for prediction (e.g., 2025): ").strip())

    # Analyze the asset
    analyze_asset(ticker, target_year)

if __name__ == "__main__":
    main()

Enter the asset ticker (e.g., AAPL, BTC-USD, VTI):  GC=F
Enter the target year for prediction (e.g., 2025):  2028


[*********************100%***********************]  1 of 1 completed

The expected percentage change range with 95% confidence for GC=F by 2028 is (-0.04837791044422102, 2.3691228892589)
